In [1]:
import pandas as pd
import numpy as np
from gensim import corpora, models, similarities, matutils
from sklearn.model_selection import train_test_split

# load data
df = pd.read_csv("../metadata/niere-example/df_classification_valid.csv")
dictionary = corpora.Dictionary.load("../metadata/niere-example/fulltitle.dict")
corpus = corpora.MmCorpus("../metadata/niere-example/fulltitle.mm")
matrix_tfidf_transpose = np.loadtxt("../metadata/niere-example/tf_idf_matrix.txt")

In [2]:
# Unsupervised method
# k-means
from scipy.cluster.vq import kmeans,vq

# computing K-Means with K = 2 (2 clusters)
centroids,_ = kmeans(matrix_tfidf_transpose,2)

# assign each sample to a cluster
idx,_ = vq(matrix_tfidf_transpose,centroids)
df['kmeans_tfidf'] = idx

In [5]:
# Compare flag with cluster results
from sklearn import metrics
from IPython.display import display

# precision = tp / (tp+fp) 
# recall = tp / (tp+fn)

display(pd.crosstab(df.kmeans_tfidf, df.useful , margins=True))
print 'Recall - the ability of the classifier to find all the positive samples: {:.1f}%'.format(metrics.recall_score(df.useful, df.kmeans_tfidf)*100)
print 'Precision - the ability of the classifier not to label as positive a sample that is negative: {:.1f}%'.format(metrics.precision_score(df.useful, df.kmeans_tfidf)*100)

useful,0,1,All
kmeans_tfidf,,,
0,577,302,879
1,3109,346,3455
All,3686,648,4334


Recall - the ability of the classifier to find all the positive samples: 53.4%
Precision - the ability of the classifier not to label as positive a sample that is negative: 10.0%


In [7]:
# Supervised method
# a) Test / Validation Split
# b) Train the logistic regression model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(matrix_tfidf_transpose, 
                                                    df.useful, test_size=0.25, random_state=0)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)


y_test_predict = classifier.predict(X_test)

display(metrics.confusion_matrix(y_test, y_test_predict))
print 'Recall - the ability of the classifier to find all the positive samples: {:.1f}%'.format(metrics.recall_score(y_test, y_test_predict)*100)
print 'Precision - the ability of the classifier not to label as positive a sample that is negative: {:.1f}%'.format(metrics.precision_score(y_test, y_test_predict)*100)

array([[927,  15],
       [124,  18]])

Recall - the ability of the classifier to find all the positive samples: 12.7%
Precision - the ability of the classifier not to label as positive a sample that is negative: 54.5%
